In [0]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

n1 = datetime.now()
print(n1)
d1 = n1.strftime('%B')
print(d1)

backd = n1 - relativedelta(years=2, months=3, days=12)
print(backd.year)

t1 = n1 - timedelta(weeks=3)
print(t1)
# 

2025-05-05 11:08:18.766257
%
2023
2025-05-05 11:08:15.766257


In [0]:
data1 = [
    (1, 101,     10, 2.5, '2024-01-15'),
(2, 105,    5,  2.5, '2024-01-28'),
(3, 102,  7,  1.2, '2024-01-10'),
(4, 106,   4,  1.2, '2024-02-01'),
(5, 103,   3,  3.0, '2024-02-15'),
(6, 104,    8,  2.5, '2024-02-20'),
(7, 107,   6,  3.0, '2024-03-05')
]

sche1 = ['sale_id', 'product_id', 'quantity', 'price', 'sale_date']

df1 = spark.createDataFrame(data1, sche1)
df1.show()

+-------+----------+--------+-----+----------+
|sale_id|product_id|quantity|price| sale_date|
+-------+----------+--------+-----+----------+
|      1|       101|      10|  2.5|2024-01-15|
|      2|       105|       5|  2.5|2024-01-28|
|      3|       102|       7|  1.2|2024-01-10|
|      4|       106|       4|  1.2|2024-02-01|
|      5|       103|       3|  3.0|2024-02-15|
|      6|       104|       8|  2.5|2024-02-20|
|      7|       107|       6|  3.0|2024-03-05|
+-------+----------+--------+-----+----------+



In [0]:
%sql
create procedure getdata()
begin

select prdo_id, date_format(sale_date, '%Y-%m') as sale_month,
sum(quantity) as total_qu, sum(price) as total_price from t1
groupby prdo_id, date_format(sale_date, '%Y-%m') orderby prdo_id, sale_month;

end;

In [0]:
%sql

-- SQL engines (like MySQL) don’t allow using an alias in the GROUP BY clause directly...

CREATE PROCEDURE calculate_monthly_sales()
BEGIN
    SELECT
        product_id,
        DATE_FORMAT(sale_date, '%Y-%m') AS sale_month,
        SUM(quantity) AS total_quantity,
        SUM(quantity * price) AS total_sales
    FROM sales_data
    GROUP BY product_id, DATE_FORMAT(sale_date, '%Y-%m')
    ORDER BY product_id, sale_month;
END;


In [0]:
from pyspark.sql.functions import *

def calculate_monthly_sales(df):
    # Create a new column 'sale_month' for month grouping
    df_monthly = df.withColumn("sale_month", date_format("sale_date", "yyyy-MM"))
    # print(df_monthly)
    # Group by product_id, product_name, and sale_month
    result = df_monthly.groupBy("product_id", "sale_month") \
        .agg(
            sum("quantity").alias("total_quantity"),
            sum(col("quantity") * col("price")).alias("total_sales")
        ) \
        .orderBy("product_id", "sale_month")
    
    return result

monthly_sales_summary = calculate_monthly_sales(df1)
monthly_sales_summary.show()


+----------+----------+--------------+-----------+
|product_id|sale_month|total_quantity|total_sales|
+----------+----------+--------------+-----------+
|       101|   2024-01|            10|       25.0|
|       102|   2024-01|             7|        8.4|
|       103|   2024-02|             3|        9.0|
|       104|   2024-02|             8|       20.0|
|       105|   2024-01|             5|       12.5|
|       106|   2024-02|             4|        4.8|
|       107|   2024-03|             6|       18.0|
+----------+----------+--------------+-----------+



In [0]:
import pandas as pd

url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'
    
df1 = pd.read_csv(url, sep = '\t')
df1.head(10)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


In [0]:
# Solution 1

df1.shape[0]  # entries <= 4622 observations

Out[30]: 4622

In [0]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB


In [0]:
df1.columns

Out[32]: Index(['order_id', 'quantity', 'item_name', 'choice_description',
       'item_price'],
      dtype='object')

In [0]:
# For the most-ordered item, how many items were ordered?

df1 = df1.groupby('item_name')
df1 = df1.sum()
df1 = df1.sort_values(['quantity'], ascending=False)
df1.head(1)

,order_id,quantity
item_name,,
Chicken Bowl,713926,761


In [0]:
# How many items were orderd in total?

t1 = df1.quantity.sum()
t1

Out[37]: 4972

In [0]:
rev = (df1['quantity'] * df1['item_price']).sum()
rev

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File /databricks/python/lib/python3.9/site-packages/pandas/core/indexes/base.py:3621, in Index.get_loc(self, key, method, tolerance)
   3620 try:
-> 3621     return self._engine.get_loc(casted_key)
   3622 except KeyError as err:

File /databricks/python/lib/python3.9/site-packages/pandas/_libs/index.pyx:136, in pandas._libs.index.IndexEngine.get_loc()

File /databricks/python/lib/python3.9/site-packages/pandas/_libs/index.pyx:163, in pandas._libs.index.IndexEngine.get_loc()

File pandas/_libs/hashtable_class_helper.pxi:5198, in pandas._libs.hashtable.PyObjectHashTable.get_item()

File pandas/_libs/hashtable_class_helper.pxi:5206, in pandas._libs.hashtable.PyObjectHashTable.get_item()

KeyError: 'item_price'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most r

In [0]:
# How many orders were made in the period?

os1 = df1.order_id.value_counts().count()
os1

Out[41]: 50

In [0]:
# How many different items are sold?

s01 = df1.item_name.value_counts().count()
s01

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-290789362603183>:3
      1 # How many different items are sold?
----> 3 s01 = df1.item_name.value_counts().count()
      4 s01

File /databricks/python/lib/python3.9/site-packages/pandas/core/generic.py:5575, in NDFrame.__getattr__(self, name)
   5568 if (
   5569     name not in self._internal_names_set
   5570     and name not in self._metadata
   5571     and name not in self._accessors
   5572     and self._info_axis._can_hold_identifiers_and_holds_name(name)
   5573 ):
   5574     return self[name]
-> 5575 return object.__getattribute__(self, name)

AttributeError: 'DataFrame' object has no attribute 'item_name'

In [0]:
# Sort by the name of the item

df1.item_name.sort_values()
 
# OR

df1.sort_values(by = "item_name")

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-290789362603189>:3
      1 # Sort by the name of the item
----> 3 df1.item_name.sort_values()
      5 # OR
      7 df1.sort_values(by = "item_name")

File /databricks/python/lib/python3.9/site-packages/pandas/core/generic.py:5575, in NDFrame.__getattr__(self, name)
   5568 if (
   5569     name not in self._internal_names_set
   5570     and name not in self._metadata
   5571     and name not in self._accessors
   5572     and self._info_axis._can_hold_identifiers_and_holds_name(name)
   5573 ):
   5574     return self[name]
-> 5575 return object.__getattribute__(self, name)

AttributeError: 'DataFrame' object has no attribute 'item_name'